# Logistic Regression
逻辑回归模型是一种常用的分类模型，它通过sigmoid或者softmax函数，将函数值映射到(0, 1)区间内，从而实现对样本的分类。在这个小作业中，你需要实现：
1. 二分类和多分类两种逻辑回归模型
2. 分别含有 L1 和 L2 两种正则项的损失函数，并计算对应的梯度
3. 权重参数W的更新
4. 比较不同的学习率对损失函数和分类器性能的影响
5. 比较不同的正则项参数对于分类器性能的影响

In [1]:
!pip install sklearn

Failed to build sklearn
  Running setup.py install for sklearn ... done


## 一、二分类逻辑回归：
### 1.1数据集介绍
这个任务中使用的数据集是手写数字集MNIST，它有50000个训练样本和10000个测试样本，共10个类别。在二分类任务上，我们对MNIST数据集进行了一个采样，抽取了数据集中的‘5’和‘3’对应的样本作为二分类的正负样本，共得到10842个训练样本，1784个测试样本，其中正负样本数量均相同。为了让大家对于这个数据集有一个更直观的认识，我们从正负样本中各抽取了8个样例进行了可视化。

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


def load_data1(path):
    # load all MNIST data
    fd = open(os.path.join(path, 'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    train_X_all = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)
    fd = open(os.path.join(path, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    train_Y_all = loaded[8:].reshape(60000).astype(np.float)
    fd = open(os.path.join(path, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    test_X_all = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)
    fd = open(os.path.join(path, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    test_Y_all = loaded[8:].reshape(10000).astype(np.float)

    #subsample data
    train_idxs_df = pd.read_csv(os.path.join(path, 'train_indices.csv'))
    test_idxs_df = pd.read_csv(os.path.join(path, 'test_indices.csv'))
    pos_train_indices = train_idxs_df['pos_train_indices'].tolist()
    neg_train_indices = train_idxs_df['neg_train_indices'].tolist()
    pos_test_indices = test_idxs_df['pos_test_indices'].tolist()
    neg_test_indices = test_idxs_df['neg_test_indices'].tolist()
    train_Y_all[pos_train_indices] = 1
    train_Y_all[neg_train_indices] = 0
    test_Y_all[pos_test_indices] = 1
    test_Y_all[neg_test_indices] = 0
    train_indices = np.append(pos_train_indices, neg_train_indices)
    test_indices = np.append(pos_test_indices, neg_test_indices)
    train_X = train_X_all[train_indices]
    train_Y = train_Y_all[train_indices]
    test_X = test_X_all[test_indices]
    test_Y = test_Y_all[test_indices]

    #visualiza data
    sample_num = 8
    pos_sample_indices = np.random.choice(pos_train_indices, sample_num, replace=False)
    neg_sample_indices = np.random.choice(neg_train_indices, sample_num, replace=False)
    for i, idx in enumerate(pos_sample_indices):
        plt_idx = i + 1
        plt.subplot(2, sample_num, plt_idx)
        plt.imshow(train_X_all[idx, :, :, :].reshape((28, 28)), cmap=plt.cm.gray)
        plt.axis('off')
        if i == 0:
            plt.title('Positive')

    for i, idx in enumerate(neg_sample_indices):
        plt_idx = sample_num + i + 1
        plt.subplot(2, sample_num, plt_idx)
        plt.imshow(train_X_all[idx, :, :, :].reshape((28, 28)), cmap=plt.cm.gray)
        plt.axis('off')
        if i == 0:
            plt.title('Negative')
    
    # reshaple into rows and normaliza
    train_X = train_X.reshape((train_X.shape[0], -1))
    test_X = test_X.reshape((test_X.shape[0], -1))
    mean_image = np.mean(train_X, axis=0)
    train_X = train_X - mean_image
    test_X = test_X - mean_image

    # add a bias columu into X
    train_X = np.hstack([train_X, np.ones((train_X.shape[0], 1))])
    test_X = np.hstack([test_X, np.ones((test_X.shape[0], 1))])
    return train_X, train_Y, test_X, test_Y


X_train, Y_train, X_test, Y_test = load_data1('/home/kesci/input/MNIST_dataset4284')

### 1.2逻辑回归模型
在这一部分中你需要完成以下内容：
1. train函数中权重的更新
2. L1和L2两种正则化的损失函数及对应梯度的计算
3. predict函数中的预测类别的计算

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class LinearRegression1(object):
    def __init__(self):
        self.W = None
    
    def train(self, X, Y, learning_rate=1e-3, reg=1e-5, reg_type='L2', num_iters=2000,
          batch_size=128, display = True, new_model = True):
        num_train, feat_dim = X.shape
        if new_model :
            self.W = 0.001 * np.random.randn(feat_dim)
        loss_history = []
        for i in range(num_iters):
            batch_indices = np.random.choice(num_train, batch_size, replace=True)
            X_batch = X[batch_indices]
            Y_batch = Y[batch_indices]
            if reg_type == 'L1':
                loss, grad = self.l1_loss(X_batch, Y_batch, reg)
            else:
                loss, grad = self.l2_loss(X_batch, Y_batch, reg)
            loss_history.append(loss)
            
            #TODO:update W
            self.W -= learning_rate * grad

            #End of your code
            
            if display and i % 100 == 0:
                print("In iteration {}/{} , the loss is {}".format(i, num_iters, loss))
        return loss_history

    def calc_loss_grad(self, X, Y):
        batch_size, dim = X.shape

        z = sigmoid(np.dot(X, np.transpose(self.W)))
        loss = -1 * (np.dot(np.squeeze(Y), np.log(z)) + np.dot((1 - np.squeeze(Y)), np.log(1 - z)))
        w_grad = np.sum(-1 * X * (np.squeeze(Y) - z).reshape((batch_size,1)), axis=0)
        return loss / batch_size , w_grad / batch_size

    
    def l1_loss(self, X, Y, reg):
        # loss = 0
        # grad = None
        
        #TODO: Calculate the loss and gradient

        loss, grad = self.calc_loss_grad(X, Y)
        loss = loss + reg * np.sum(abs(self.W))
        grad = grad + reg

        # End of your code
        
        return loss, grad
    
    def l2_loss(self, X, Y, reg):
        loss = 0
        grad = None
        
        #TODO: Calculate the loss and gradient
        
        loss, grad = self.calc_loss_grad(X, Y)
        loss = loss + reg * np.sum(self.W * self.W)
        grad = grad + 2 * reg * self.W
        
        # End of your code
        
        return loss, grad
    
    def predict(self, X, threshold=0.5):
        Y_pred = np.zeros(X.shape[0])

        #TODO: Calculate the prediction
        # Y_pred = sigmoid(np.dot(X, self.W))
        for i in range(X.shape[0]):
            ans = sigmoid(np.sum(self.W * X[i]))
            if (ans >= threshold):
                Y_pred[i] = 1
            else:
                Y_pred[i] = 0;
        
        #End of your code
        
        return Y_pred
        
    def draw_W(self):
        Xmin = np.min(self.W)
        Xmax = np.max(self.W)
        plt.hist(self.W,bins=50)
        plt.xlabel('Area')
        plt.xlim(Xmin, Xmax)
        plt.ylabel('Number')
        # plt.ylim(0,100)
        plt.title('W')
        plt.show()

### 1.3 训练模型实例
在这一部分，你不需要完成任何代码，你可以通过这一部分验证你上面实现的LogisticRegression1的代码是否实现正确。

In [8]:
lr_param = 1.5e-6
reg_param = 0.01

model = LinearRegression1()
loss_history = model.train(X_train, Y_train, lr_param, reg_param, 'L2')
pred = model.predict(X_test)
acc = np.mean(pred == Y_test)
print("The Accuracy is {}\n".format(acc))

model.draw_W()

x = range(len(loss_history))
plt.plot(x, loss_history, label='Loss')
plt.legend()
plt.xlabel('Iteration Num')
plt.ylabel('Loss')
plt.show()
W = model.W

In iteration 0/2000 , the loss is 0.8994590522777538
In iteration 100/2000 , the loss is 0.17534203681480814
In iteration 200/2000 , the loss is 0.12262025762205832
In iteration 300/2000 , the loss is 0.11586265532277332
In iteration 400/2000 , the loss is 0.1751349401935428
In iteration 500/2000 , the loss is 0.1191054906240259
In iteration 600/2000 , the loss is 0.06237859137078771
In iteration 700/2000 , the loss is 0.07289062233622569
In iteration 800/2000 , the loss is 0.14250482917110485
In iteration 900/2000 , the loss is 0.12832735809462315
In iteration 1000/2000 , the loss is 0.12392580940045864
In iteration 1100/2000 , the loss is 0.18576922639877577
In iteration 1200/2000 , the loss is 0.16336939296353306
In iteration 1300/2000 , the loss is 0.1318461928838651
In iteration 1400/2000 , the loss is 0.11914072393778617
In iteration 1500/2000 , the loss is 0.11592232373477075
In iteration 1600/2000 , the loss is 0.13956918988974104
In iteration 1700/2000 , the loss is 0.18826003

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

### 1.4 学习率和Loss函数、模型性能的关系
因为学习率和正则化参数都是超参数，在一般的训练过程中，我们没办法直接优化，所以我们一般会将训练集细分成训练集和验证集，然后通过模型在验证集上的表现选择一个最优的超参数，再将它对应的最优的模型应用到测试集中。
在这一部分你需要完成以下内容：
1. 尝试多种不同的学习率
2. 储存学习率对应的损失函数值到L1_loss和L2_loss中（我们对损失函数值进行了20步平均化处理）。
3. 储存学习率对应的**在验证集上**的正确率到L1_lr_val_acc和L2_lr_val_acc中

#### 注意：
因为已有代码中L1_loss，L1_lr_val_acc都是数组，在可视化的过程中我们需要学习率和它们相对应，比如learning_rates[0]对应的loss和validation accuracy应该储存在数组index为0的位置

#### 拓展：
在这个部分中采取的损失函数都是定值，如果你有时间的话，可以尝试根据迭代轮数改变学习率，并比较不变的学习率和变化的学习率对于模型性能的影响。

In [24]:
reg = 0.01
reg_types = ['L1', 'L2']
L1_loss = []
L2_loss = []
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)
L1_lr_val_acc = []
L2_lr_val_acc = []

#TODO:
learning_rates = [7e-5, 5e-5, 3e-5, 1e-5, 7e-6, 5e-6, 3e-6, 1e-6, 7e-7]

for reg_type in range(len(reg_types)):
    for i in range(len(learning_rates)):
        model = LinearRegression1()
        learning_rate = learning_rates[i]
        loss_history = model.train(X_train, Y_train, learning_rate, reg, reg_types[reg_type], display = False)
        accuracy = np.mean(model.predict(X_val) == Y_val)
        
        if reg_type == 0:
            L1_loss.append(loss_history)
            L1_lr_val_acc.append(accuracy)
        else:
            L2_loss.append(loss_history)
            L2_lr_val_acc.append(accuracy)
            
        print(reg_types[reg_type], learning_rate, accuracy)
#End of your code

#visulize the relationship between lr and loss(L1)
for i, lr in enumerate(learning_rates):
    L1_loss_label = str(lr) + 'L1'
    # L2_loss_label = str(lr) + 'L2'
    L1_loss_i = L1_loss[i]
    # L2_loss_i = L2_loss[i]
    ave_L1_loss = np.zeros_like(L1_loss_i)
    # ave_L2_loss = np.zeros_like(L2_loss_i)
    ave_step = 20
    for j in range(len(L1_loss_i)):
        if j < ave_step:
            ave_L1_loss[j] = np.mean(L1_loss_i[0: j + 1])
            # ave_L2_loss[j] = np.mean(L2_loss_i[0: j + 1])
        else:
            ave_L1_loss[j] = np.mean(L1_loss_i[j - ave_step + 1: j + 1])    
            # ave_L2_loss[j] = np.mean(L2_loss_i[j - ave_step + 1: j + 1])
    x = range(len(L1_loss_i))
    plt.plot(x, ave_L1_loss, label=L1_loss_label)
    # plt.plot(x, ave_L2_loss, label=L2_loss_label)
    
plt.legend()
plt.xlabel('high-parameter lr')
plt.ylabel('Loss')
plt.show()

#visulize the relationship between lr and loss(L2)
for i, lr in enumerate(learning_rates):
    # L1_loss_label = str(lr) + 'L1'
    L2_loss_label = str(lr) + 'L2'
    # L1_loss_i = L1_loss[i]
    L2_loss_i = L2_loss[i]
    # ave_L1_loss = np.zeros_like(L1_loss_i)
    ave_L2_loss = np.zeros_like(L2_loss_i)
    ave_step = 20
    for j in range(len(L1_loss_i)):
        if j < ave_step:
            # ave_L1_loss[j] = np.mean(L1_loss_i[0: j + 1])
            ave_L2_loss[j] = np.mean(L2_loss_i[0: j + 1])
        else:
            # ave_L1_loss[j] = np.mean(L1_loss_i[j - ave_step + 1: j + 1])    
            ave_L2_loss[j] = np.mean(L2_loss_i[j - ave_step + 1: j + 1])
    x = range(len(L2_loss_i))
    # plt.plot(x, ave_L1_loss, label=L1_loss_label)
    plt.plot(x, ave_L2_loss, label=L2_loss_label)
    
plt.legend()
plt.xlabel('high-parameter lr')
plt.ylabel('Loss')
plt.show()

#visulize the relationship between lr and accuracy
x = range(len(learning_rates))
plt.plot(x, L1_lr_val_acc, label='L1_val_acc')
plt.plot(x, L2_lr_val_acc, label='L2_val_acc')
plt.xticks(x, learning_rates)
plt.margins(0.08)
plt.legend()
plt.xlabel('high-parameter lr')
plt.ylabel('Validation Accuracy')
plt.show()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log


L1 7e-05 0.9493670886075949
L1 5e-05 0.9479606188466948
L1 3e-05 0.9521800281293952
L1 1e-05 0.9549929676511955
L1 7e-06 0.9549929676511955
L1 5e-06 0.9549929676511955
L1 3e-06 0.9479606188466948
L1 1e-06 0.939521800281294
L1 7e-07 0.9409282700421941
L2 7e-05 0.9479606188466948
L2 5e-05 0.9507735583684951
L2 3e-05 0.9493670886075949
L2 1e-05 0.9507735583684951
L2 7e-06 0.9507735583684951
L2 5e-06 0.9535864978902954
L2 3e-06 0.9521800281293952
L2 1e-06 0.9409282700421941
L2 7e-07 0.9381153305203939


<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

#### Question1: 学习率和损失函数的变化、模型性能之间分别有什么关系？
Answer1:由图表中可以看到，对于不同的学习率，损失函数均是先极速下降后平缓下降，最后在一定范围内震荡变化。同时，从图表中可以看到，损失函数最后收敛的值基本符合随着学习率的减小而增大的规律（这一规律在学习率较大的情况下较不明显，在学习率较小的情况下较为明显）。（由于图像线条过于密集，故将L1和L2的结果分开表示）。
而模型的性能随着学习率的变化有所变化。整体呈现上凸形，即在一定范围内模型性能更好，当学习率高于或低于这一范围，则模型性能会较为明显地下降。从图表中可以看到对于L1和L2两种回归模型，这一范围大概为$[5e-06,3e-05]$这一区间。

#### 拓展
以下代码尝试了根据迭代轮数改变学习率
变化的学习率可以一定程度上做到比不变的学习率的准确性更高，而且这需要更为精确的对轮次和学习率的把握。

In [49]:
L2_altr_loss = []
learning_rates_step = [1e-5, 8e-6, 6e-6, 5e-6]
num_iter = 500
model = LinearRegression1()
for i in range(len(learning_rates_step)):
    learning_rate = learning_rates_step[i]
    if i == 0:
        L2_altr_loss += model.train(X_train, Y_train, learning_rate, reg, 'L2',
            num_iters = num_iter, display = False, new_model = True)
    else:
        L2_altr_loss += model.train(X_train, Y_train, learning_rate, reg, 'L2',
            num_iters = num_iter, display = False, new_model = False)

    accuracy_altr = np.mean(model.predict(X_val) == Y_val)
    print("accuracy_altr now is {}".format(accuracy_altr))
    

accuracy = np.mean(model.predict(X_val) == Y_val)
print("Total accuracy is {}".format(accuracy))
#visulize l2_altr_loss
x = range(len(L2_altr_loss))
plt.plot(x, L2_altr_loss, label='Loss')
plt.legend()
plt.xlabel('Iteration Num')
plt.ylabel('Loss')
plt.show()

accuracy_altr now is 0.9626373626373627
accuracy_altr now is 0.967032967032967
accuracy_altr now is 0.9692307692307692
accuracy_altr now is 0.967032967032967
Total accuracy is 0.967032967032967


<Figure size 432x288 with 1 Axes>

### 1.5 正则项与模型性能
在这一部分中，你需要完成以下内容：
1. 尝试多个正则化参数的值
2. 储存对应的在**验证集上**的正确率到L1_reg_val_acc和L2_reg_val_acc中
3. 通过验证集X_val和Y_val选择最优的正则化超参数，并储存最优正则化参数和对应模型

已有的代码会画出正则化参数和验证集上正确率的关系图，并计算最优的模型在测试集上的正确率。

#### 注意：
和上面学习率一样，L1_reg_val_acc的存储也需要和正则化参数值对应。

In [26]:
learning_rate = 1.5e-6
reg_types = ['L1', 'L2']
L1_reg_val_acc = []
L2_reg_val_acc = []
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)
best_L1_model = None
best_L2_model = None
best_L1_reg = 0
best_L2_reg = 0

#TODO
regs = [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035]

#End of your code
for reg_type in range(len(reg_types)):
    best_accuracy = 0
    for i in range(len(regs)):
        model = LinearRegression1()
        reg = regs[i]
        loss_history = model.train(X_train, Y_train, learning_rate, reg, reg_types[reg_type], display = False)
        accuracy = np.mean(model.predict(X_val) == Y_val)

        if reg_type == 0:
            L1_reg_val_acc.append(accuracy)
            if (accuracy > best_accuracy):
                best_accuracy = accuracy
                best_L1_model = model
                best_L1_reg = reg
        else:
            L2_reg_val_acc.append(accuracy)
            if (accuracy > best_accuracy):
                best_accuracy = accuracy
                best_L2_model = model
                best_L2_reg = reg
        print(reg_types[reg_type], reg, accuracy)

#visuliza the relation of regularization parameter and validation accuracy
x = range(len(regs))
plt.plot(x, L1_reg_val_acc, label='L1_val_acc')
plt.plot(x, L2_reg_val_acc, label='L2_val_acc')
plt.xticks(x, regs)
plt.margins(0.08)
plt.legend()
plt.xlabel('high-parameter reg')
plt.ylabel('Validation Accuracy')
plt.show()

#Compute the performance of best model on the test set
L1_pred = best_L1_model.predict(X_test)
L1_acc = np.mean(L1_pred == Y_test)
print("The Accuracy with L1 regularization parameter {} is {}\n".format(best_L1_reg, L1_acc))
L2_pred = best_L2_model.predict(X_test)
L2_acc = np.mean(L2_pred == Y_test)
print("The Accuracy with L2 regularization parameter {} is {}\n".format(best_L2_reg, L2_acc))

L1 0.001 0.9494505494505494
L1 0.005 0.9582417582417583
L1 0.01 0.9560439560439561
L1 0.015 0.9560439560439561
L1 0.02 0.9560439560439561
L1 0.025 0.9538461538461539
L1 0.03 0.9626373626373627
L1 0.035 0.9494505494505494
L2 0.001 0.9494505494505494
L2 0.005 0.9538461538461539
L2 0.01 0.9560439560439561
L2 0.015 0.9560439560439561
L2 0.02 0.9538461538461539
L2 0.025 0.9538461538461539
L2 0.03 0.9516483516483516
L2 0.035 0.9472527472527472


<Figure size 432x288 with 1 Axes>

The Accuracy with L1 regularization parameter 0.03 is 0.9540358744394619

The Accuracy with L2 regularization parameter 0.01 is 0.9568385650224215



## 二、多分类逻辑回归

### 2.1 加载数据集

In [9]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


def load_data2(path):
    # load all MNIST data
    fd = open(os.path.join(path, 'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    train_X = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)
    fd = open(os.path.join(path, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    train_Y = loaded[8:].reshape(60000).astype(np.float)
    fd = open(os.path.join(path, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    test_X = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)
    fd = open(os.path.join(path, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    test_Y = loaded[8:].reshape(10000).astype(np.float)

    #visualiza data
    sample_num = 8
    num_classes = 10
    for y in range(num_classes):
        idxs = np.flatnonzero(train_Y == y)
        idxs = np.random.choice(idxs, sample_num, replace=False)
        for i, idx in enumerate(idxs):
            plt_idx = i * num_classes + y + 1
            plt.subplot(sample_num, num_classes, plt_idx)
            plt.imshow(train_X[idx, :, :, :].reshape((28,28)),cmap=plt.cm.gray)
            plt.axis('off')
            if i == 0:
                plt.title(y)
    plt.show()

    # reshaple into rows and normaliza
    train_X = train_X.reshape((train_X.shape[0], -1))
    test_X = test_X.reshape((test_X.shape[0], -1))
    mean_image = np.mean(train_X, axis=0)
    train_X = train_X - mean_image
    test_X = test_X - mean_image

    # add a bias columu into X
    train_X = np.hstack([train_X, np.ones((train_X.shape[0], 1))])
    test_X = np.hstack([test_X, np.ones((test_X.shape[0], 1))])
    train_Y = train_Y.astype(np.int32)
    test_Y = test_Y.astype(np.int32)
    return train_X, train_Y, test_X, test_Y


X_train, Y_train, X_test, Y_test = load_data2('/home/kesci/input/MNIST_dataset4284')

<Figure size 432x288 with 80 Axes>

### 2.2逻辑回归模型
在这一部分中你需要完成与二分类逻辑回归相同的任务。

In [10]:
def softmax(x, axis=1):
    row_max = x.max(axis=axis)
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    return x_exp / x_sum
    
class LinearRegression2(object):
    def __init__(self):
        self.W = None
    def train(self, X, Y, learning_rate=1e-3, reg=1e-5, reg_type='L2', num_iters=2000,
              batch_size=128, display = True, new_model = True):
        num_train, feat_dim = X.shape
        num_classes = 10
        if new_model:
            self.W = 0.001 * np.random.randn(feat_dim, num_classes)
        loss_history = []
        for i in range(num_iters):
            batch_indices = np.random.choice(num_train, batch_size, replace=True)
            X_batch = X[batch_indices]
            Y_batch = Y[batch_indices]
            if reg_type == 'L1':
                loss, grad = self.l1_loss(X_batch, Y_batch, reg)
            else:
                loss, grad = self.l2_loss(X_batch, Y_batch, reg)
            loss_history.append(loss)
            
            #TODO: update W
            self.W -= learning_rate * grad
            
            #End of your code
       
            if display and i % 100 == 0:
                print("In iteration {}/{} , the loss is {}".format(i, num_iters, loss))
        if display :
            print(self.W)
        return loss_history
        
    def calc_loss_grad(self, X, Y):
        m = X.shape[0]
        A = softmax(np.dot(X, self.W))
        # expand the Y
        Y_ = np.zeros((X.shape[0], self.W.shape[1]))
        Y_[np.arange(X.shape[0]), Y] = 1
        J = -1 / m * np.sum(Y_*np.log(A))
        dw = -1 / m * np.dot(X.T, (Y_-A))
        return J, dw

    def l1_loss(self, X, Y, reg):
        # loss = 0
        # grad = None
    
        #TODO: Calculate the loss and gradient
        loss, grad = self.calc_loss_grad(X, Y)
        loss = loss + reg * np.sum(abs(self.W))
        grad = grad + reg
        
        # End of your code
    
        return loss, grad
    
    def l2_loss(self, X, Y, reg):
        # loss = 0
        # grad = None
        
        #TODO: Calculate the loss and gradient
        loss, grad = self.calc_loss_grad(X, Y)
        loss = loss + reg * np.sum(self.W*self.W)
        grad = grad + 2 * reg * self.W
        # print(loss, grad)
        
        # End of your code
        
        return loss, grad

    def predict(self, X):
        # Y_pred = np.zeros(X.shape[0])

        #TODO: Calculate the prediction
        Y_ = softmax(np.dot(X,self.W))
        max_index = np.argmax(Y_, axis = 1)
        Y_pred = max_index
        # print(Y_pred)
        #End of your code
    
        return Y_pred
        
    def draw_W(self):
        Xmin = np.min(self.W)
        Xmax = np.max(self.W)
        plt.hist(self.W,bins=50)
        plt.xlabel('Area')
        plt.xlim(Xmin, Xmax)
        plt.ylabel('Number')
        # plt.ylim(0,100)
        plt.title('W')
        plt.show()


### 2.3 训练模型样例
在这一部分，你不需要完成任何代码，你可以通过这一部分验证你上面实现的LogisticRegression1的代码是否实现正确。

In [75]:
lr_param = 1e-6
reg_param = 0.01
model = LinearRegression2()
loss_history = model.train(X_train, Y_train, lr_param, reg_param, 'L2')
pred = model.predict(X_test)
acc = np.mean(pred == Y_test)
print("The Accuracy is {}\n".format(acc))

model.draw_W()

x = range(len(loss_history))
plt.plot(x, loss_history, label='Loss')
plt.legend()
plt.xlabel('Iteration Num')
plt.ylabel('Loss')
plt.show()

In iteration 0/2000 , the loss is 3.2323281955687992
In iteration 100/2000 , the loss is 1.0594463210560192
In iteration 200/2000 , the loss is 0.6874427680540208
In iteration 300/2000 , the loss is 0.5513659178329648
In iteration 400/2000 , the loss is 0.5450278846430155
In iteration 500/2000 , the loss is 0.6171831804741016
In iteration 600/2000 , the loss is 0.6017910654986751
In iteration 700/2000 , the loss is 0.35459624822015273
In iteration 800/2000 , the loss is 0.4138831578860036
In iteration 900/2000 , the loss is 0.3435582489815632
In iteration 1000/2000 , the loss is 0.39111284805454793
In iteration 1100/2000 , the loss is 0.5267107269526045
In iteration 1200/2000 , the loss is 0.3814061947092372
In iteration 1300/2000 , the loss is 0.3501531322019915
In iteration 1400/2000 , the loss is 0.40526082029707644
In iteration 1500/2000 , the loss is 0.3275881127016504
In iteration 1600/2000 , the loss is 0.3226397983707609
In iteration 1700/2000 , the loss is 0.31426820704635733


<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

### 2.4 学习率与损失函数、模型性能的关系
因为学习率和正则化参数都是超参数，在一般的训练过程中，我们没办法直接优化，所以我们一般会将训练集细分成训练集和验证集，然后通过模型在验证集上的表现选择一个最优的超参数，再将它对应的最优的模型应用到测试集中。
在这一部分你需要完成以下内容：
1. 尝试多种不同的学习率
2. 储存学习率对应的损失函数值到L1_loss和L2_loss中（我们对损失函数值进行了20步平均化处理）。
3. 储存学习率对应的**在验证集上**的正确率到L1_lr_val_acc和L2_lr_val_acc中

#### 注意：
因为已有代码中L1_loss，L1_lr_val_acc都是数组，在可视化的过程中我们需要学习率和它们相对应，比如learning_rates[0]对应的loss和validation accuracy应该储存在数组index为0的位置

#### 拓展：
在这个部分中采取的损失函数都是定值，如果你有时间的话，可以尝试根据迭代轮数改变学习率，并比较不变的学习率和变化的学习率对于模型性能的影响。

In [56]:
reg = 0.01
reg_types = ['L1', 'L2']
L1_loss = []
L2_loss = []
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)
L1_lr_val_acc = []
L2_lr_val_acc = []

#TODO
learning_rates = [7e-5, 5e-5, 3e-5, 1e-5, 7e-6, 5e-6, 3e-6, 1e-6, 7e-7]

for reg_type in range(len(reg_types)):
    for i in range(len(learning_rates)):
        model = LinearRegression2()
        learning_rate = learning_rates[i]
        loss_history = model.train(X_train, Y_train, learning_rate, reg, reg_types[reg_type], display = False)
        accuracy = np.mean(model.predict(X_val) == Y_val)
        
        if reg_type == 0:
            L1_loss.append(loss_history)
            L1_lr_val_acc.append(accuracy)
        else:
            L2_loss.append(loss_history)
            L2_lr_val_acc.append(accuracy)
            
        print(reg_types[reg_type], learning_rate, accuracy)

#End of your code

#visulize the relationship between lr and loss
for i, lr in enumerate(learning_rates):
    L1_loss_label = str(lr) + 'L1'
    L2_loss_label = str(lr) + 'L2'
    L1_loss_i = L1_loss[i]
    L2_loss_i = L2_loss[i]
    ave_L1_loss = np.zeros_like(L1_loss_i)
    ave_L2_loss = np.zeros_like(L2_loss_i)
    ave_step = 20
    for j in range(len(L1_loss_i)):
        if j < ave_step:
            ave_L1_loss[j] = np.mean(L1_loss_i[0: j + 1])
            ave_L2_loss[j] = np.mean(L2_loss_i[0: j + 1])
        else:
            ave_L1_loss[j] = np.mean(L1_loss_i[j - ave_step + 1: j + 1])    
            ave_L2_loss[j] = np.mean(L2_loss_i[j - ave_step + 1: j + 1])
    x = range(len(L1_loss_i))
    plt.plot(x, ave_L1_loss, label=L1_loss_label)
    plt.plot(x, ave_L2_loss, label=L2_loss_label)
    
plt.legend()
plt.xlabel('high-parameter lr')
plt.ylabel('Loss')
plt.show()

#visulize the relationship between lr and accuracy
x = range(len(learning_rates))
plt.plot(x, L1_lr_val_acc, label='L1_val_acc')
plt.plot(x, L2_lr_val_acc, label='L2_val_acc')
plt.xticks(x, learning_rates)
plt.margins(0.08)
plt.legend()
plt.xlabel('high-parameter lr')
plt.ylabel('Validation Accuracy')
plt.show()

L1 7e-05 0.9039166666666667
L1 5e-05 0.9075
L1 3e-05 0.9126666666666666
L1 1e-05 0.9121666666666667
L1 7e-06 0.9116666666666666
L1 5e-06 0.9081666666666667
L1 3e-06 0.903
L1 1e-06 0.88975
L1 7e-07 0.8829166666666667
L2 7e-05 0.9021666666666667
L2 5e-05 0.9126666666666666
L2 3e-05 0.912
L2 1e-05 0.9114166666666667
L2 7e-06 0.9101666666666667
L2 5e-06 0.9091666666666667
L2 3e-06 0.90525
L2 1e-06 0.8889166666666667
L2 7e-07 0.8838333333333334


<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

#### 拓展
以下代码尝试了根据迭代轮数改变学习率.
变化的学习率可以一定程度上做到比不变的学习率的准确性更高，并且这种变化在多分类中比二分类中更为明显。同时，不同阶段选取的学习率对最后的准确性也有较大的影响。最后经过多次尝试，选择了这一组准确性最高的学习率。

In [64]:
reg = 0.01
L2_altr_loss = []
learning_rates_step = [5e-5, 7e-6, 3e-5, 1e-5]
num_iter = 500
model = LinearRegression2()
for i in range(len(learning_rates_step)):
    learning_rate = learning_rates_step[i]
    if i == 0:
        L2_altr_loss += model.train(X_train, Y_train, learning_rate, reg, 'L2',
            num_iters = num_iter, display = False, new_model = True)
    else:
        L2_altr_loss += model.train(X_train, Y_train, learning_rate, reg, 'L2',
            num_iters = num_iter, display = False, new_model = False)

    accuracy_altr = np.mean(model.predict(X_val) == Y_val)
    print("accuracy_altr now is {}".format(accuracy_altr))
    
accuracy = np.mean(model.predict(X_val) == Y_val)
print("Total accuracy is {}".format(accuracy))
#visulize l2_altr_loss
x = range(len(L2_altr_loss))
plt.plot(x, L2_altr_loss, label='Loss')
plt.legend()
plt.xlabel('Iteration Num')
plt.ylabel('Loss')
plt.show()

accuracy_altr now is 0.9118333333333334
accuracy_altr now is 0.9225833333333333
accuracy_altr now is 0.9209166666666667
accuracy_altr now is 0.9241666666666667
Total accuracy is 0.9241666666666667


<Figure size 432x288 with 1 Axes>

### 2.5 正则项与模型性能
在这一部分中，你需要完成以下内容：
1. 尝试多个正则化参数的值
2. 储存对应的在**验证集上**的正确率到L1_reg_val_acc和L2_reg_val_acc中
3. 通过验证集X_val和Y_val选择最优的正则化超参数，并储存最优正则化参数和对应模型

已有的代码会画出正则化参数和验证集上正确率的关系图，并计算最优的模型在测试集上的正确率。

#### 注意：
和上面学习率一样，L1_reg_val_acc的存储也需要和正则化参数值对应。

In [12]:
learning_rate = 1.5e-6
reg_types = ['L1', 'L2']
L1_reg_val_acc = []
L2_reg_val_acc = []
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)
best_L1_model = None
best_L2_model = None
best_L1_reg = 0
best_L2_reg = 0

#TODO
regs = [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04]
for reg_type in range(len(reg_types)):
    best_accuracy = 0
    for i in range(len(regs)):
        model = LinearRegression2()
        reg = regs[i]
        loss_history = model.train(X_train, Y_train, learning_rate, reg, reg_types[reg_type], display = False)
        accuracy = np.mean(model.predict(X_val) == Y_val)

        if reg_type == 0:
            L1_reg_val_acc.append(accuracy)
            if (accuracy > best_accuracy):
                best_accuracy = accuracy
                best_L1_model = model
                best_L1_reg = reg
        else:
            L2_reg_val_acc.append(accuracy)
            if (accuracy > best_accuracy):
                best_accuracy = accuracy
                best_L2_model = model
                best_L2_reg = reg
        print(reg_types[reg_type], reg, accuracy)

#End of your code


#visuliza the relation of regularization parameter and validation accuracy
x = range(len(regs))
plt.plot(x, L1_reg_val_acc, label='L1_val_acc')
plt.plot(x, L2_reg_val_acc, label='L2_val_acc')
plt.xticks(x, regs)
plt.margins(0.08)
plt.legend()
plt.xlabel('high-parameter reg')
plt.ylabel('Validation Accuracy')
plt.show()

#Compute the performance of best model on the test set
L1_pred = best_L1_model.predict(X_test)
L1_acc = np.mean(L1_pred == Y_test)
print("The Accuracy with L1 regularization parameter {} is {}\n".format(best_L1_reg, L1_acc))
L2_pred = best_L2_model.predict(X_test)
L2_acc = np.mean(L2_pred == Y_test)
print("The Accuracy with L2 regularization parameter {} is {}\n".format(best_L2_reg, L2_acc))

L1 0.001 0.8933333333333333
L1 0.005 0.8933333333333333
L1 0.01 0.8940625
L1 0.015 0.8958333333333334
L1 0.02 0.89625
L1 0.025 0.8923958333333334
L1 0.03 0.8946875
L1 0.035 0.895
L1 0.04 0.8959375
L2 0.001 0.8925
L2 0.005 0.8934375
L2 0.01 0.8945833333333333
L2 0.015 0.895
L2 0.02 0.8934375
L2 0.025 0.8960416666666666
L2 0.03 0.8971875
L2 0.035 0.8947916666666667
L2 0.04 0.8944791666666667


<Figure size 432x288 with 1 Axes>

The Accuracy with L1 regularization parameter 0.02 is 0.9001

The Accuracy with L2 regularization parameter 0.03 is 0.8995



#### Question2: 对于上面的多分类逻辑回归模型，你觉得它的权重矩阵数值上会呈现出什么样子？你可以通过可视化的方法观察权重矩阵。
Answer2:权重矩阵数值的绝对值都较小，绝对值中最大的大概在$10^{-3}$这一量级。根据**1.3**和**2.3**部分代码对W矩阵数值分析后显示的直方图，其分布于sigmoid函数的导函数图像类似，可见softmax和sigmoid函数对W矩阵数值分布的影响。